In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [3]:
def get_title(data):
    data['Title']=data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
    return data
train=get_title(train)
test=get_title(test)

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [5]:
columns=['Name','Ticket','Cabin']
train.drop(columns=columns,inplace=True)
test.drop(columns=columns,inplace=True)

In [6]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
Title            0
dtype: int64

In [7]:
test.isnull().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Fare            1
Embarked        0
Title           0
dtype: int64

In [8]:
train['Embarked'].fillna(train['Embarked'].mode()[0],inplace=True)
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         0
Title            0
dtype: int64

In [9]:
train.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], dtype=object)

In [10]:
train['Title'].value_counts()

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Major             2
Mlle              2
Col               2
Don               1
Ms                1
the Countess      1
Capt              1
Sir               1
Mme               1
Jonkheer          1
Lady              1
Name: Title, dtype: int64

In [11]:
titles_m={
    "Capt": "Rare",
    "Col": "Rare",
    "Major": "Rare",
    "Jonkheer": "Rare",
    "Don": "Rare",
    "Dona": "Miss",
    "Sir" : "Rare",
    "Dr": "Rare",
    "Rev": "Rare",
    "the Countess":"Rare",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Rare"
}

In [12]:
train.Title=train.Title.map(titles_m)
test.Title=test.Title.map(titles_m)

In [13]:
train.Title.value_counts()

Mr        517
Miss      184
Mrs       127
Master     40
Rare       23
Name: Title, dtype: int64

In [14]:
test.isnull().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Fare            1
Embarked        0
Title           0
dtype: int64

In [15]:
test['Embarked'].fillna(test['Embarked'].mode()[0],inplace=True)

In [16]:
#cl=test.loc[test['Fare'].isnull(),['Pclass']]
#mean=train[train['Pclass']==3].mean()
#train['Fare'].fillna(mean,inplace=True)
train.shape

(891, 10)

In [17]:
#train.drop('PassengerId',axis=1,inplace=True)

train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         0
Title            0
dtype: int64

In [18]:
grouped_median_train = train.groupby(['Sex','Pclass','Title']).median()
grouped_median_train = grouped_median_train.reset_index()[['Sex', 'Pclass', 'Title', 'Age']]
def fill_age(row):
    condition = (
        (grouped_median_train['Sex'] == row['Sex']) & 
        (grouped_median_train['Title'] == row['Title']) & 
        (grouped_median_train['Pclass'] == row['Pclass'])
    ) 
    if np.isnan(grouped_median_train[condition]['Age'].values):
        condition = (
            (grouped_median_train['Sex'] == row['Sex']) & 
            (grouped_median_train['Pclass'] == row['Pclass'])
        )

    return grouped_median_train[condition]['Age'].values


def process_age(data):
    
    data.Age = data.apply(lambda row: fill_age(row) if np.isnan(row['Age']) else row['Age'], axis=1)
    return data

train = process_age(train)


In [19]:
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Title          0
dtype: int64

In [20]:
test=process_age(test)
test.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           1
Embarked       0
Title          0
dtype: int64

In [21]:
test[test['Fare'].isnull()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
152,1044,3,male,60.5,0,0,NaN,S,Mr


In [22]:
m=train[train['Pclass']==3]['Fare'].mean()
test.Fare.fillna(m,inplace=True)
test.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Title          0
dtype: int64

In [23]:
train=train.drop('PassengerId',axis=1)

In [24]:
def family(data):
    data['Family']=data['Parch']+data['SibSp']+1
    return data
train=family(train)
test=family(test)

In [25]:
train = train.drop(['SibSp'], axis = 1)
test  = test.drop(['SibSp'], axis = 1)

In [26]:
sex_map = {"male": 1, "female": 0}
embark_map={"S": 1, "Q": 2, "C": 3}
title_map={"Mr": 1, "Master": 2, "Mrs": 3,"Miss": 4, "Rare": 5}
train.Sex=train.Sex.map(sex_map)
train.Embarked=train.Embarked.map(embark_map)
train.Title=train.Title.map(title_map)
test.Sex=test.Sex.map(sex_map)
test.Embarked=test.Embarked.map(embark_map)
test.Title=test.Title.map(title_map)

In [27]:
train

,Survived,Pclass,Sex,Age,Parch,Fare,Embarked,Title,Family
0,0,3,1,22,0,7.2500,1,1,2
1,1,1,0,38,0,71.2833,3,3,2
2,1,3,0,26,0,7.9250,1,4,1
3,1,1,0,35,0,53.1000,1,3,2
4,0,3,1,35,0,8.0500,1,1,1
...,...,...,...,...,...,...,...,...,...
886,0,2,1,27,0,13.0000,1,5,1
887,1,1,0,19,0,30.0000,1,4,1
888,0,3,0,[18.0],2,23.4500,1,4,4
889,1,1,1,26,0,30.0000,3,1,1


In [28]:
y=train['Survived']
train=train.drop('Survived',axis=1)
X=train


In [29]:
X.to_csv('cleaned_train.csv',index=False)
test.to_csv('cleaned_test.csv',index=False)
y.to_csv('train_target.csv',index=False)

In [36]:
from sklearn.model_selection import StratifiedKFold,GridSearchCV,cross_val_score
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
params = {
    'n_estimators':[100,200,400,500,800],'max_features':['auto','sqrt','log2'],'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']}
gs1=GridSearchCV(estimator=rfc,param_grid=params,cv=10,n_jobs=-1)

In [37]:
gs1.fit(X,y)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             ii

In [38]:
gs1.best_params_


{'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'log2',
 'n_estimators': 100}

In [39]:
gs1.best_score_

0.8406285072951739

In [44]:
rfc1=RandomForestClassifier(n_estimators=100,max_features='log2',criterion='gini',max_depth=10)
rfc1.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [45]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(y,rfc1.predict(X))
acc

0.9461279461279462

In [50]:
passengerID=test['PassengerId']
test.drop('PassengerId',axis=1,inplace=True)

In [51]:
y_pred=rfc1.predict(test)
y_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [53]:
result=pd.DataFrame({"PassengerId":passengerID , "Survived":y_pred})
result.to_csv('rfc1.csv',index=False)